In [1]:
import matplotlib.pyplot as plt
from milestone1.question4 import *
from etape2_Q3 import *
import numpy as np
from sklearn.calibration import CalibrationDisplay
from matplotlib.gridspec import GridSpec

In [2]:
directory2015 = r'data_saved/play_by_play/2015/regular'
directory2016 = r'data_saved/play_by_play/2016/regular'
directory2017 = r'data_saved/play_by_play/2017/regular'
directory2018 = r'data_saved/play_by_play/2018/regular'

In [3]:
df2015 = create_full_df(directory2015)
df2016 = create_full_df(directory2016)
df2017 = create_full_df(directory2017)
df2018 = create_full_df(directory2018)

In [4]:
df_train = pd.concat([df2015, df2016, df2017], ignore_index=True)
df_val = df2018
# df_train = df_train.dropna(subset="Distance")
# df_val = df_val.dropna(subset="Distance")

In [5]:
X_train = df_train[['Secondes_jeu','Period_Number','X_Coordinate','Y_Coordinate','Distance','Angle','Filet_vide','Last_event_type'
,'X_last_event','Y_last_event','Sec_from_lastEvent','Dis_from_lastEvent','Rebond','Angle_change','Vitesse']]
y_train = df_train["Est_un_but"]

X_val = df_val[['Secondes_jeu','Period_Number','X_Coordinate','Y_Coordinate','Distance','Angle','Filet_vide','Last_event_type'
,'X_last_event','Y_last_event','Sec_from_lastEvent','Dis_from_lastEvent','Rebond','Angle_change','Vitesse']]
y_val = df_val["Est_un_but"]

#creer une dictionnaire pour transformer les events en event_id, car le model ne peut pas etre entraine sur les donnees str
dict_event_id = {'Unknown': 1, 'Faceoff': 2, 'Hit': 3,'Giveaway': 4, 'Goal': 5, 'Shot': 6,'Missed Shot': 7, 'Penalty': 8, 'Stoppage': 9,
'Sub': 10, 'Fight': 11, 'Takeaway': 12, 'Blocked Shot': 13, 'Period Start': 14,'Period End': 15, 'Game End': 16, 'Game Scheduled': 17,
'Period Ready': 18, 'Period Official': 19, 'Shootout Complete': 20 ,'Early Intermission Start': 21, 'Early Intermission End': 22, 'Game Official': 23,
'Official Challenge': 24, 'Emergency Goaltender': 25, 'Failed Shot Attempt': 26}

#triatement les nan, les nans sont changee par 0
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)

#transformer les events en event_id
X_train['Last_event_type'] = X_train['Last_event_type'].apply(lambda x : dict_event_id[x])
X_val['Last_event_type'] = X_val['Last_event_type'].apply(lambda x : dict_event_id[x])




In [6]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

clf.fit(X_train,y_train)
y_prob = clf.predict_proba(X_val)
score = clf.score(X_val, y_val)
print("accuracy: ",score)



accuracy:  0.9034430392352121


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
from sklearn import svm

clf = svm.SVC(probability=True)
clf.fit(X_train, y_train)
y_prob = clf.predict_proba(X_val)
score = clf.score(X_val, y_val)
